In [ ]:
%matplotlib inline

# Extension Principle and Fuzzy Arithmetic

This is a Jupyter page to help you explore the Extension Principle

Have fun!

Below, I support three binary functions, 

* A+B
* max(A,B)
* min(A,B)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

# our domain
X = np.asarray( range(1,30,1) )

# fuzzy set 1
# about 3
A = np.zeros( X.size )
A[0] = 0.
A[1] = 0.5
A[2] = 1.0
A[3] = 0.5
A[4] = 0.

# fuzzy set 2
# about 7
B = np.zeros( X.size )
B[4] = 0.
B[5] = 0.5
B[6] = 1.0
B[7] = 0.5
B[8] = 0.

# output fuzzy set
C = np.zeros( X.size )

# define the function
def BinaryEpFunction( arg1, arg2, WhichOp ):
    if( WhichOp == 1 ):
        return (arg1 + arg2)       # sum
    elif( WhichOp == 2 ):
        return max( arg1, arg2 )   # max
    elif( WhichOp == 3 ):
        return min( arg1, arg2 )   # min
    elif( WhichOp == 4 ):
        return arg1 * arg2         # mul
    
# which operator do we want to use? 1 == sum, 2 == max, 3 == min, 4 == mul
WhichOp = 1
    
N = X.size
for i in range(N): # go over each point in our output domain
    for j in range(N): # go over all x
        for k in range(N): # go over all y
            r = BinaryEpFunction( X[j], X[k], WhichOp )
            if( r == X[i] ): # is this one of our points?
                m = min( A[j], B[k] ) # t-norm
                if( m > C[i] ):
                    C[i] = m          # we are taking the sup
                    
# plot it!
fig = plt.figure(figsize=(14, 6))
plt.stem(A,markerfmt='ko')
plt.stem(B,markerfmt='ro')
plt.stem(C,markerfmt='gx')
b=range(0,N)
plt.fill_between( b, C, color="skyblue", alpha=0.4)
xi = list(range(len(X)))
plt.xticks(xi, X)
if( WhichOp == 1 ):
    plt.title('Set [A+B](y)')
elif( WhichOp == 2 ):
    plt.title('Set [max(A,B)](y)')
elif( WhichOp == 3 ):
    plt.title('Set [min(A,B)](y)')
elif( WhichOp == 4 ):
    plt.title('Set [A*B](y)')    
plt.grid()
plt.show()

# Q&A

Think about

* Unary operations
* Does max(A,B) give you what you thought...? What do you think the max(A,B) should be?

# What about an alpha cut approach?

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

# first input
# about 3
A = np.asarray( [1,3,5] )

# second input
# about 7
B = np.asarray( [5,7,9] )

# alpha cut parameters
NoSlices = 30
AlphaCuts = np.asarray( range(1,NoSlices+1) ).astype(float) / NoSlices

# define the function
def IntervalFunction( a, b, c, d, WhichOp ):
    if( WhichOp == 1 ): # sum
        return (a+c), (b+d)        
    elif( WhichOp == 2 ): # mul
        return min(a*c,min(a*d,min(b*c,b*d))), max(a*c,max(a*d,max(b*c,b*d)))    
    elif( WhichOp == 3 ): # minus
        return (a-d), (b-c)  
    elif( WhichOp == 4 ): # div
        return min(a/c,min(a/d,min(b/c,b/d))), max(a/c,max(a/d,max(b/c,b/d)))   
    
# get interval from a triangular membership function
def GetTriInterval( fuzzyset, alphacut ):
    # memb == (x-a)/(b-a)
    #  alphacut = (x-fuzzyset[0]) / (fuzzyset[1]-fuzzyset[0])
    #  (fuzzyset[1]-fuzzyset[0]) * alphacut = x-fuzzyset[0]
    #  (fuzzyset[1]-fuzzyset[0]) * alphacut + fuzzyset[0] = x
    a = (fuzzyset[1]-fuzzyset[0]) * alphacut + fuzzyset[0]
    # memb == (c-x)/(c-b)
    #  alphacut = (fuzzyset[2]-x) / (fuzzyset[2]-fuzzyset[1])
    #  (fuzzyset[2]-fuzzyset[1]) * alphacut = (fuzzyset[2]-x)
    #  fuzzyset[2] - (fuzzyset[2]-fuzzyset[1]) * alphacut = x    
    b = fuzzyset[2] - (fuzzyset[2]-fuzzyset[1]) * alphacut
    return a, b

# which operator do we want to use? 1 == sum, 2 == mul, 3 == minus, 4 = div
WhichOp = 1
    
fig = plt.figure(figsize=(14, 6))    
for i in range( AlphaCuts.size ):
    alpha = AlphaCuts[i]
    # get interval 1
    a, b = GetTriInterval( A, alpha )
    plt.plot([a,b],[alpha,alpha],'--c')
    # get interval 2
    c, d = GetTriInterval( B, alpha )
    plt.plot([c,d],[alpha,alpha],'--g')    
    # calc result
    e, f = IntervalFunction( a, b, c, d, WhichOp )
    plt.plot([e,f],[alpha,alpha],'r') 

plt.grid()
plt.show()